In [1]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/barcoded_flu_pdmH1N1/lib/python3.8/site-packages', '/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\xc6<\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluHA-A_S1_L001_R1_001.fastq.gz\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluHA-B_S2_L001_R1_001.fastq.gz\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluNA-A_S17_L001_R1_001.fastq.gz\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluNA-B_S18_L001_R1_001.fastq.gz\x94\x8c\x8e/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluHA-A_S3_L001_R1_001.fastq.gz\x94\x8c\x8e/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluHA-B_S4_L001_R1_001.fastq.gz\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluNA-A_S19_L001_R1_001.fastq.gz\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluNA-B_S20_L001_R1_001.fastq.gz\x94\x8c\x90/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluHA-A_S5_L001_R1_001.fastq.gz\x94\x8c\x90/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluHA-B_S6_L001_R1_001.fastq.gz\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluNA-A_S21_L001_R1_001.fastq.gz\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluNA-B_S22_L001_R1_001.fastq.gz\x94\x8c\x94/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluHA-A_S7_L001_R1_001.fastq.gz\x94\x8c\x94/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluHA-B_S8_L001_R1_001.fastq.gz\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluNA-A_S23_L001_R1_001.fastq.gz\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluNA-B_S24_L001_R1_001.fastq.gz\x94\x8c(results/viral_fastq10x/viral_bc_locs.csv\x94\x8c\x1edata/flu_sequences/flu-CA09.gb\x94\x8c,notebooks/viral_barcodes_in_progeny.py.ipynb\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\nfastq_list\x94K\x00K\x10\x86\x94\x8c\rviral_bc_locs\x94K\x10N\x86\x94\x8c\rviral_genbank\x94K\x11N\x86\x94\x8c\x08notebook\x94K\x12N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh*\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h0)}\x94\x8c\x05_name\x94h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bh h\x06\x8c\tNamedlist\x94\x93\x94)\x81\x94(h\nh\x0bh\x0ch\rh\x0eh\x0fh\x10h\x11h\x12h\x13h\x14h\x15h\x16h\x17h\x18h\x19e}\x94(h\x1e}\x94h(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bubh"h\x1ah$h\x1bh&h\x1cub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8cKresults/viral_progeny/scProgenyProduction_trial1_viral_bc_in_progeny.csv.gz\x94a}\x94(h\x1e}\x94\x8c\x17viral_bc_in_progeny_csv\x94K\x00N\x86\x94sh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bhOhLub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94(\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x0cBlockManager\x94\x93\x94)\x81\x94(]\x94(\x8c\x18pandas.core.indexes.base\x94\x8c\n_new_Index\x94\x93\x94hi\x8c\x05Index\x94\x93\x94}\x94(\x8c\x04data\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x07\x85\x94hs\x8c\x05dtype\x94\x93\x94\x8c\x02O8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01|\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?t\x94b\x89]\x94(\x8c\nexperiment\x94\x8c\x06source\x94\x8c\x03tag\x94\x8c\x04gene\x94\x8c\treplicate\x94\x8c\x03run\x94\x8c\nfastq_path\x94et\x94b\x8c\x04name\x94Nu\x86\x94R\x94hk\x8c\x1bpandas.core.indexes.numeric\x94\x8c\nInt64Index\x94\x93\x94}\x94(hohrhuK\x00\x85\x94hw\x87\x94R\x94(K\x01K\x10\x85\x94h|\x8c\x02i8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C\x80\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x94t\x94bh\x8bNu\x86\x94R\x94e]\x94hrhuK\x00\x85\x94hw\x87\x94R\x94(K\x01K\x07K\x10\x86\x94h\x7f\x89]\x94(\x8c\x1ascProgenyProduction_trial1\x94h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5h\xa5\x8c\x0bsupernatant\x94h\xa6h\xa6h\xa6h\xa6h\xa6h\xa6h\xa6\x8c\x10second_infection\x94h\xa7h\xa7h\xa7h\xa7h\xa7h\xa7h\xa7\x8c\x02wt\x94h\xa8h\xa8h\xa8\x8c\x03syn\x94h\xa9h\xa9h\xa9\x8c\x02wt\x94h\xaah\xaah\xaa\x8c\x03syn\x94h\xabh\xabh\xab\x8c\x05fluHA\x94h\xac\x8c\x05fluNA\x94h\xad\x8c\x05fluHA\x94h\xae\x8c\x05fluNA\x94h\xaf\x8c\x05fluHA\x94h\xb0\x8c\x05fluNA\x94h\xb1\x8c\x05fluHA\x94h\xb2\x8c\x05fluNA\x94h\xb3\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x0breplicate_1\x94\x8c\x0breplicate_2\x94\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluHA-A_S1_L001_R1_001.fastq.gz\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluHA-B_S2_L001_R1_001.fastq.gz\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluNA-A_S17_L001_R1_001.fastq.gz\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluNA-B_S18_L001_R1_001.fastq.gz\x94\x8c\x8e/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluHA-A_S3_L001_R1_001.fastq.gz\x94\x8c\x8e/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluHA-B_S4_L001_R1_001.fastq.gz\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluNA-A_S19_L001_R1_001.fastq.gz\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluNA-B_S20_L001_R1_001.fastq.gz\x94\x8c\x90/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluHA-A_S5_L001_R1_001.fastq.gz\x94\x8c\x90/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluHA-B_S6_L001_R1_001.fastq.gz\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluNA-A_S21_L001_R1_001.fastq.gz\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluNA-B_S22_L001_R1_001.fastq.gz\x94\x8c\x94/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluHA-A_S7_L001_R1_001.fastq.gz\x94\x8c\x94/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluHA-B_S8_L001_R1_001.fastq.gz\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluNA-A_S23_L001_R1_001.fastq.gz\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluNA-B_S24_L001_R1_001.fastq.gz\x94et\x94ba]\x94hkhm}\x94(hohrhuK\x00\x85\x94hw\x87\x94R\x94(K\x01K\x07\x85\x94h\x7f\x89]\x94(h\x83h\x84h\x85h\x86h\x87h\x88h\x89et\x94bh\x8bNu\x86\x94R\x94a}\x94\x8c\x060.14.1\x94}\x94(\x8c\x04axes\x94hh\x8c\x06blocks\x94]\x94}\x94(\x8c\x06values\x94h\xa2\x8c\x08mgr_locs\x94\x8c\x08builtins\x94\x8c\x05slice\x94\x93\x94K\x00K\x07K\x01\x87\x94R\x94uaust\x94b\x8c\x04_typ\x94\x8c\tdataframe\x94\x8c\t_metadata\x94]\x94\x8c\x05attrs\x94}\x94ubK\x10K"K\x00]\x94(\x8c\x05fluHA\x94\x8c\x05fluNA\x94ee}\x94(h\x1e}\x94(\x8c\x08fastq_df\x94K\x00N\x86\x94\x8c\x14viral_barcode_length\x94K\x01N\x86\x94\x8c\x1dviral_barcode_upstream_length\x94K\x02N\x86\x94\x8c viral_barcode_mismatch_threshold\x94K\x03N\x86\x94\x8c\x14barcoded_viral_genes\x94K\x04N\x86\x94uh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bj,\x01\x00\x00haj.\x01\x00\x00K\x10j0\x01\x00\x00K"j2\x01\x00\x00K\x00j4\x01\x00\x00j\'\x01\x00\x00ub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x1ascProgenyProduction_trial1\x94a}\x94(h\x1e}\x94\x8c\x04expt\x94K\x00N\x86\x94sh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94b\x8c\x04expt\x94jC\x01\x00\x00ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01e}\x94(h\x1e}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94uh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bjY\x01\x00\x00K\x01j[\x01\x00\x00K\x01ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8cJresults/logs/scProgenyProduction_trial1_viral_barcodes_in_progeny.py.ipynb\x94a}\x94(h\x1e}\x94h&K\x00N\x86\x94sh(]\x94(h*h+eh*h.h0\x85\x94R\x94(h0)}\x94h4h*sNt\x94bh+h.h0\x85\x94R\x94(h0)}\x94h4h+sNt\x94bh&jj\x01\x00\x00ub\x8c\x06config\x94}\x94(\x8c\x08max_cpus\x94K\x10\x8c\x07log_dir\x94\x8c\x0cresults/logs\x94\x8c\x0cfastq10x_dir\x94\x8c\x10results/fastq10x\x94\x8c\x0emkfastq10x_dir\x94\x8c\x1fresults/fastq10x/mkfastq_output\x94\x8c\ngenome_dir\x94\x8c\x0fresults/genomes\x94\x8c\trefgenome\x94\x8c\x19results/genomes/refgenome\x94\x8c\x14aligned_fastq10x_dir\x94\x8c\x18results/aligned_fastq10x\x94\x8c\x12viral_fastq10x_dir\x94\x8c\x16results/viral_fastq10x\x94\x8c\x1bviral_tags_bcs_in_cells_dir\x94\x8c\x1fresults/viral_tags_bcs_in_cells\x94\x8c\x11viral_progeny_dir\x94\x8c\x15results/viral_progeny\x94\x8c\x0fcell_genome_ftp\x94\x8cmftp://ftp.ensembl.org/pub/release-98/fasta/canis_familiaris/dna/Canis_familiaris.CanFam3.1.dna.toplevel.fa.gz\x94\x8c\x0ccell_gtf_ftp\x94\x8c^ftp://ftp.ensembl.org/pub/release-98/gtf/canis_familiaris/Canis_familiaris.CanFam3.1.98.gtf.gz\x94\x8c\x0cviral_genome\x94\x8c!data/flu_sequences/flu-CA09.fasta\x94\x8c\tviral_gtf\x94\x8c\x1fdata/flu_sequences/flu-CA09.gtf\x94\x8c\rviral_genbank\x94\x8c\x1edata/flu_sequences/flu-CA09.gb\x94\x8c\x14viral_tag_identities\x94\x8c+data/flu_sequences/flu-CA09_viral_tags.yaml\x94\x8c\x0escoreGapNoncan\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapGCAG\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapATAC\x94J\xfc\xff\xff\xff\x8c\x14cb_whitelist_10x_url\x94\x8ckhttps://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/3M-february-2018.txt.gz\x94\x8c\x10cb_whitelist_10x\x94\x8c-results/aligned_fastq10x/cb_whitelist_10x.txt\x94\x8c\ncb_len_10x\x94K\x10\x8c\x0bumi_len_10x\x94K\x0c\x8c"infection_calling_by_viral_tag_fdr\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x14viral_barcode_length\x94K\x10\x8c\x1dviral_barcode_upstream_length\x94K"\x8c viral_barcode_mismatch_threshold\x94K\x00\x8c\x0bexperiments\x94}\x94(\x8c\x0ehashing_lowMOI\x94}\x94(\x8c\x0bdescription\x94X*\x01\x00\x00Single-cell transcriptomics using the wildtype and dblySyn viral tag variants. Infection volume was chosen based on the results of `hashing_trial1` and flow cytometry. The incolum volume for the wildtype tag variant was about 12-fold higher, and for the dblSyn tag variant was about 24-fold higher.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-W8urOmOAQ7L6U4HAXMNy\x94\x8c\rexpect_ncells\x94M\x88\x13\x8c\x0ftranscriptomics\x94}\x94(h\xc6C\x04\x07\xe4\x06\x02\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B3\x94\x8c\nbcl_folder\x94\x8cG/home/solexa/ngs/illumina/bloom_lab/200602_M03100_0578_000000000-J3464/\x94\x8c\x04lane\x94\x8c\x01*\x94uh\xc6C\x04\x07\xe4\x07\x01\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B3\x94\x8c\nbcl_folder\x94\x8c^/shared/ngs/illumina/agreaney/200701_D00300_0988_BHGFGMBCX3/raw/200701_D00300_0988_BHGFGMBCX3/\x94\x8c\x04lane\x94K\x02uh\xc6C\x04\x07\xe4\x08\x1a\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B3\x94\x8c\nbcl_folder\x94\x8ci/shared/ngs/illumina/bloom_lab/bloom_lab/200826_D00300_1040_BHHGLWBCX3/raw/200826_D00300_1040_BHHGLWBCX3/\x94\x8c\x04lane\x94j\xb9\x01\x00\x00uuu\x8c\x0fhashing_highMOI\x94}\x94(\x8c\x0bdescription\x94\x8c\x9aSingle-cell transcriptomics using the wildtype and dblSyn viral tag variants at a high MOI. Infection volume was based on the results of `hashing_trial2`.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-ToK66qT0Mfmf6FsexKo4\x94\x8c\rexpect_ncells\x94M\x88\x13\x8c\x0ftranscriptomics\x94}\x94(h\xc6C\x04\x07\xe4\x07\x18\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B5\x94\x8c\nbcl_folder\x94\x8cB/shared/ngs/illumina/bloom_lab/200724_M03100_0593_000000000-J33YK/\x94\x8c\x04lane\x94j\xb9\x01\x00\x00uh\xc6C\x04\x07\xe4\x08\x1a\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B5\x94\x8c\nbcl_folder\x94\x8ci/shared/ngs/illumina/bloom_lab/bloom_lab/200826_D00300_1040_BHHGLWBCX3/raw/200826_D00300_1040_BHHGLWBCX3/\x94\x8c\x04lane\x94j\xb9\x01\x00\x00uu\x8c\x17pacbio_viral_sequencing\x94}\x94\x8c\x1c2020-10-20_all_segments_run1\x94\x8cq/fh/fast/bloom_j/SR/ngs/pacbio/201019_BernadetaDadonaite/r54228_20201020_192758/m54228_201020_194205.subreads.bam\x94suh\xa5}\x94(\x8c\x0bdescription\x94\x8c\xe7Single-cell transcriptomics performed on low MOI-infected sample. Innoculum volumes based on `hashing_trial2` data. Loaded all collected cells from two 24-well wells, each infected with either wildtype or dblySn viral tag variants.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-20lOO7l1FdEJJIpxV6vD\x94\x8c\rexpect_ncells\x94M\x10\'\x8c\x0ftranscriptomics\x94}\x94h\xc6C\x04\x07\xe4\t\x19\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B8\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/200925_D00300_1065_AHHL7NBCX3/raw/200925_D00300_1065_AHHL7NBCX3/\x94\x8c\x04lane\x94K\x02us\x8c\x0eviral_barcodes\x94}\x94(h\xa6}\x94(h\xa8}\x94(h\xac}\x94(h\xb4}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xf7sh\xb5}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xf8suh\xad}\x94(h\xb6}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xf9sh\xb7}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xfasuuh\xa9}\x94(h\xae}\x94(h\xb8}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xfbsh\xb9}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xfcsuh\xaf}\x94(h\xba}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xfdsh\xbb}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xfesuuuh\xa7}\x94(h\xaa}\x94(h\xb0}\x94(h\xbc}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94h\xffsh\xbd}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94j\x00\x01\x00\x00suh\xb1}\x94(h\xbe}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94j\x01\x01\x00\x00sh\xbf}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94j\x02\x01\x00\x00suuh\xab}\x94(h\xb2}\x94(h\xc0}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94j\x03\x01\x00\x00sh\xc1}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94j\x04\x01\x00\x00suh\xb3}\x94(h\xc2}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94j\x05\x01\x00\x00sh\xc3}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94j\x06\x01\x00\x00suuuuu\x8c\x1ascProgenyProduction_trial2\x94}\x94(\x8c\x0bdescription\x94\x8c\xe7Single-cell transcriptomics performed on low MOI-infected sample. Innoculum volumes based on `hashing_trial2` data. Loaded all collected cells from two 24-well wells, each infected with either wildtype or dblySn viral tag variants.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-73UD4Mx8uXn1xcxlV1ym\x94\x8c\rexpect_ncells\x94M\x10\'\x8c\x0ftranscriptomics\x94}\x94h\xc6C\x04\x07\xe4\t\x19\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B9\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/200925_D00300_1065_AHHL7NBCX3/raw/200925_D00300_1065_AHHL7NBCX3/\x94\x8c\x04lane\x94j\xb9\x01\x00\x00us\x8c\x0eviral_barcodes\x94}\x94(\x8c\x0bsupernatant\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluHA-A_S9_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluHA-B_S10_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluNA-A_S25_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluNA-B_S26_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluHA-A_S11_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluHA-B_S12_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluNA-A_S27_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluNA-B_S28_L001_R1_001.fastq.gz\x94suuu\x8c\x10second_infection\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluHA-A_S13_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluHA-B_S14_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluNA-A_S29_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluNA-B_S30_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluHA-A_S15_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluHA-B_S16_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluNA-A_S31_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xc6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluNA-B_S32_L001_R1_001.fastq.gz\x94suuuuuuu\x8c\x04rule\x94\x8c\x19viral_barcodes_in_progeny\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c[/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir('/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing');
######## snakemake preamble end #########


# Extract viral barcodes from progeny viral barcode sequencing data
This Python Jupyter notebook parses viral barcodes from progeny sequencing data sources (e.g. supernatant or second infection). For each sequencing sample, the viral barcodes are parsed and tallied.

Import Python modules:

In [2]:
import pandas as pd

import Bio.SeqIO

Get `snakemake` variables [as described here](https://snakemake.readthedocs.io/en/stable/snakefiles/rules.html#jupyter-notebook-integration):

In [3]:
viral_bc_locs = snakemake.input.viral_bc_locs
viral_genbank = snakemake.input.viral_genbank
fastq_df = snakemake.params.fastq_df
viral_barcode_length = snakemake.params.viral_barcode_length
viral_barcode_upstream_length = snakemake.params.viral_barcode_upstream_length
viral_barcode_mismatch_threshold = snakemake.params.viral_barcode_mismatch_threshold
barcoded_viral_genes = snakemake.params.barcoded_viral_genes

Read the viral barcode locations:

In [4]:
print(f"Reading viral barcode locations from {viral_bc_locs}")
bc_locs_df = pd.read_csv(viral_bc_locs)
display(bc_locs_df)

if len(bc_locs_df) != bc_locs_df['gene'].nunique():
    raise ValueError('code assumes at most one barcode per gene')

for index, row in bc_locs_df.iterrows():
    assert ((row['end'] - (row['start']-1)) == viral_barcode_length), \
           'barcode start and end sites do not match expected length'

Reading viral barcode locations from results/viral_fastq10x/viral_bc_locs.csv


,gene,start,end
0,fluHA,1828,1843
1,fluNA,1551,1566


Get the upstream sequence for each barcoded gene:

In [5]:
upstream_seqs = []
for s in Bio.SeqIO.parse(viral_genbank, 'genbank'):
    if s.id in barcoded_viral_genes:
        bc_features = [f for f in s.features if f.type == 'viral_barcode']
        assert len(bc_features) == 1
        bc_feature = bc_features[0]
        upstream_seq_start = int(bc_locs_df.query('gene == @s.id')['start']) - viral_barcode_upstream_length - 1 #Adjust indexing
        upstream_seq_end = int(bc_locs_df.query('gene == @s.id')['start']) - 1 # Exclude final nt site which is in barcode
        upstream_seqs.append((s.id,
                              str(s.seq[upstream_seq_start:upstream_seq_end])))
upstream_seqs_df = pd.DataFrame.from_records(upstream_seqs,
                                             columns=['gene', 'upstream_seq'])

display(upstream_seqs_df)

if len(upstream_seqs_df) != upstream_seqs_df['gene'].nunique():
    raise ValueError('code assumes at most one barcode per gene')

,gene,upstream_seq
0,fluHA,GATTTGCATATAATGATGCACCATAGGCGGCCGC
1,fluNA,GATAGATAAATAATAGTGCACCATAGGCGGCCGC


Iterate through sequencing samples and make a list of all FASTQ files:

In [31]:
assert len(fastq_df['experiment'].unique()) == 1, "code assumes dataframe contains 1 experiment"

for source in fastq_df['source'].unique():
    for tag in fastq_df['tag'].unique():
        for gene in fastq_df['gene'].unique():
            for replicate in fastq_df['replicate'].unique():
                print(fastq_df.query('(source == @source) and '
                                     '(tag == @tag) and '
                                     '(gene == @gene) and '
                                     '(replicate == @replicate')['fastq_path'])

TokenError: ('EOF in multi-line statement', (2, 0))

For now just touch the intended output file so `Snakefile` runs via `snakemake`:

In [6]:
from pathlib import Path

Path(snakemake.output.viral_bc_in_progeny_csv).touch()